In [1]:
import import_ipynb

In [2]:
%run NLTK.ipynb import Tokenize,LoweringStemming,BagOfWord  # Use %run to import other jupyte notebook file

In [3]:
AllWords = []
Tags = []
XY = []

XTrain = []
YTrain = []

In [4]:
import json
with open("Data.json", "r") as F:
    intents = json.load(F)
intents

{'Data': [{'tag': 'start',
   'patterns': ['Hi',
    'Hello',
    'Hey',
    'Namaste',
    'How are you?',
    'Is anyone there?',
    'Good day',
    'Hi there',
    'Hey there',
    "What's up?",
    'Howdy',
    'Greetings',
    'Morning!',
    'Afternoon!',
    'Evening!',
    'Hello ji',
    'Hiya!',
    'Yo!',
    'Whatâ€™s cooking?',
    'Howâ€™s it going?',
    'Nice to meet you!',
    'Kya haal hai?',
    'Sab theek?',
    'Hello bhai',
    'Kaise ho?'],
   'responses': ['Hello! How can I assist you today?',
    'Hi there! What can I do for you?',
    'Hey! How can I help?',
    'Namaste! What brings you here today?',
    'Hi! How can I make your day better?']},
  {'tag': 'stop',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Thanks for the help',
    'Take care',
    'Catch you later',
    'Talk to you soon',
    'Farewell',
    'Have a great day',
    'See you soon',
    'Later',
    'Iâ€™m off',
    'Iâ€™m out',
    'See ya',
    'Until next time',
    'Go

In [5]:
for intent in intents["Data"]:
    Tag = intent["tag"]
    Tags.append(Tag)
    for Pattern in intent["patterns"]:
        Words = Tokenize(Pattern)
        Words = [LoweringStemming(Stem) for Stem in Words if(Stem not in ["?","!",".",",","'s"])]
        AllWords.extend(Words)  #Words is a list
        XY.append((Words,Tag))
        print(Words)

AllWords = sorted(set(AllWords)) #Set() for remove duplicate
print("AllWords : ",AllWords)
print("XY : ",XY)

['hi']
['hello']
['hey']
['namast']
['how', 'are', 'you']
['is', 'anyon', 'there']
['good', 'day']
['hi', 'there']
['hey', 'there']
['what', 'up']
['howdi']
['greet']
['morn']
['afternoon']
['even']
['hello', 'ji']
['hiya']
['yo']
['whatâ€™', 'cook']
['howâ€™', 'it', 'go']
['nice', 'to', 'meet', 'you']
['kya', 'haal', 'hai']
['sab', 'theek']
['hello', 'bhai']
['kais', 'ho']
['bye']
['see', 'you', 'later']
['goodby']
['thank', 'for', 'the', 'help']
['take', 'care']
['catch', 'you', 'later']
['talk', 'to', 'you', 'soon']
['farewel']
['have', 'a', 'great', 'day']
['see', 'you', 'soon']
['later']
['iâ€™m', 'off']
['iâ€™m', 'out']
['see', 'ya']
['until', 'next', 'time']
['good', 'night']
['bye', 'for', 'now']
['take', 'it', 'easi']
['peac', 'out']
['adio']
['iâ€™m', 'head', 'out']
['talk', 'later']
['got', 'ta', 'go']
['see', 'you', 'around']
['be', 'well']
['where', 'can', 'i', 'find', 'a', 'good', 'north', 'indian', 'restaur']
['can', 'you', 'recommend', 'a', 'north', 'indian', 'restaur']

In [6]:
import numpy as np
for (Pattern,Tag) in XY:
    XTrain.append(BagOfWord(Pattern,AllWords))
    YTrain.append(Tags.index(Tag))  
XTrain = np.array(XTrain)
YTrain = np.array(YTrain)

In [7]:
XTrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
YTrain

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10

In [9]:
from torch.utils.data import Dataset,DataLoader
class ChatDataset(Dataset):
    # def __init__(self):
    #     self.NSample = len(XTrain)
    #     self.XData = XTrain
    #     self.YData = YTrain

    # def __getitem__(self,index):
    #     return self.XData[index],self.YData[index]

    def __init__(self):
        self.NSample = len(XTrain)
        self.XData = XTrain
        self.YData = YTrain

    def __getitem__(self, index):
        return self.XData[index], self.YData[index]

    def __len__(self):
        return self.NSample

In [10]:
BatchSize = 8
DataSet = ChatDataset()

TrainLoader = DataLoader(dataset=DataSet, batch_size=BatchSize, shuffle=True)
print(TrainLoader)

In [11]:
%run Model.ipynb import NeuralNet

In [12]:
InputSize = len(AllWords)
HiddenSize = 10
NumClasses = len(Tags)
print(InputSize, HiddenSize, NumClasses)

Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model = NeuralNet(InputSize, HiddenSize, NumClasses).to(Device)
print(Device)

129 10 15
cpu


In [13]:

LearningRate = 0.001
NumEpochs = 2000

Criterian = nn.CrossEntropyLoss()
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

for Epochs in range(NumEpochs):
    for (Words,Labels) in TrainLoader:
        Words = Words.to(Device)
        Labels = Labels.to(Device).long()    

        Output = Model(Words)
        Loss = Criterian(Output, Labels)

        Optimizer.zero_grad()
        Loss.backward()
        Optimizer.step()

    if(Epochs + 1) % 100 == 0 :
        print(f'Epoch [{Epochs+1}/{NumEpochs}], Loss: {Loss.item():.4f}')
        
print(f"Final Loss , Loss={Loss.item():.4f}")

Epoch [100/2000], Loss: 0.0021
Epoch [200/2000], Loss: 0.0001
Epoch [300/2000], Loss: 0.1229
Epoch [400/2000], Loss: 0.0000
Epoch [500/2000], Loss: 0.1187
Epoch [600/2000], Loss: 0.2311
Epoch [700/2000], Loss: 0.1158
Epoch [800/2000], Loss: 0.1153
Epoch [900/2000], Loss: 0.0000
Epoch [1000/2000], Loss: 0.2311
Epoch [1100/2000], Loss: 0.1164
Epoch [1200/2000], Loss: 0.1160
Epoch [1300/2000], Loss: 0.1152
Epoch [1400/2000], Loss: 0.0000
Epoch [1500/2000], Loss: 0.0000
Epoch [1600/2000], Loss: 0.1155
Epoch [1700/2000], Loss: 0.0000
Epoch [1800/2000], Loss: 0.2311
Epoch [1900/2000], Loss: 0.1162
Epoch [2000/2000], Loss: 0.0000
Final Loss , Loss=0.0000


In [14]:
data = {
"model_state": Model.state_dict(),
"input_size": InputSize,
"hidden_size": HiddenSize,
"output_size": NumClasses,
"all_words": AllWords,
"tags": Tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
